converte Folhas de pagamento recebidas em formato html para planilhas excell
registrando cada evento como uma linha

# Configuração do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import html
import os
import re
from tqdm.notebook import tqdm_notebook

In [2]:
try:
  from unidecode import unidecode
except:
  !pip install unidecode
  from unidecode import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.3 MB/s eta 0:00:00


# Processamento e Leitura das Folhas de Pagamento (por grupo)

## Declaração de Funções gerais

In [ ]:
def clean_html(path:str) -> list[str]:
  with open(path, 'r', encoding='latin-1') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')
  # Salva apenas as linhas que tem 'class'
  # e aplica a formatação para adicionar acentos
  arq_html = [html.unescape(row) for row in arq_html if 'class' in row or 'CLASS' in row]
  # Remove as divs
  for i, row in enumerate(arq_html):
    pattern = r'<[^>]*>'
    new_row = re.sub(pattern, '', row)
    # Em algum
    new_row = new_row.replace('\xa0', ' ')
    arq_html[i] = new_row

  return arq_html

In [ ]:
# Padroniza os campos textuais da folha, removendo
# acentos, cedilhas e espaços no começo e fim
# E formata os CNPJs
def padroniza(folha:pd.DataFrame) -> pd.DataFrame:

  def padroniza_palavra(word):
    word = unidecode(str(word))
    word = word.replace('ç', 'c')
    word = word.replace('Ç', 'C')
    word = word.strip()
    return word

  colunas = ['Empregador', 'Nome_Empregado', 'Cargo', 'Lotação']

  for coluna in colunas:
    folha[coluna] = folha[coluna].apply(padroniza_palavra)

  def ajusta_cnpj(cnpj):
    cnpj = str(cnpj)
    caracteres = [',', '.', '-', '/']
    for char in caracteres:
      cnpj = cnpj.replace(char, '')
    return f'{int(cnpj):014}'

  folha['CNPJ'] = folha['CNPJ'].apply(ajusta_cnpj)

  def ajusta_lotacao(lotacao):
    if lotacao[0].isnumeric():
      lotacao = lotacao[lotacao.find(' - ')+3:]
    return lotacao

  folha['Lotação'] = folha['Lotação'].apply(ajusta_lotacao)

  folha = folha.loc[(folha['Código_Evento'] != '') | (folha['Desc_Evento'] != '')]

  for i in range(folha.shape[0]):
    if folha['Empregador'].iloc[i] == 'S SERVICOS LTDA' and folha['Desc_Evento'].iloc[i] == 'Pensão Alimentícia':
      folha['Código_Evento'].iloc[i] = 225

  if 'index' in folha.columns:
    folha = folha.drop('index', axis='columns')


  condonal = [
      'CONDAP PRESTACAO DE SERVICOS EM C',
      'CONDAP SERVICOS CONDOMINAIS LTDA',
      'CONDLIMP PREST. SERVICO CONDOMINIO',
      'CONDLIMP PRESTACAO S. COND. LTDA',
      'CONDO LOCACAO DE MAO DE OBRA LTD',
      'CONDO SERVICOS DE APOIO ADMINISTRA',
      'CONDONAL COMERCIO E SERVICOS LTDA',
      'CONDONAL SERVICOS E ADMINISTRACAO',
      'CONDOS ADMINISTRADORA DE CONDOMI',
      'ESSENCIAL CONDOMINIOS LTDA',
      'ESSENCIAL GESTAO E SERVICOS LTDA']

  remover = [
      '1/3 Abono Pecuniário Fér', '1/3 Férias', '1/3 Férias Rescisão', '13o Indenizado Rescisão',
      '13o Indenizado Rescisão', '13o Salário Complementar', '13o Salário Proporc.Resc',
      '13o Salário Resc.Mater.', 'Abono Pecuniário Férias', 'Adic.Noturno 13o Inden.',
      'Adic.Noturno 13o Prop.', 'Adic.Noturno Férias', 'Aviso Prévio Indenizado',
      'Aviso Prévio Reavido', 'Bolsa Estágio Rescisão', 'Desc. Cesta Basica Resc.',
      'Desc.13o Salário Adto', 'Descto.Adto.13ºResc.s/Pag', 'Desconto Adto Férias',
      'Diferença de Férias', 'Estabilidade Rescisão','FGTS 13o Salário','Férias',
      'Férias Proporc.Rescisão', 'Férias Vencidas Rescisão', 'INSS 13o Salário', 'IRRF 13o Salário',
      'IRRF Férias', 'Ind.T.Contr.Adic.Insalubr', 'Ind.T.Contr.Adic.Noturno',
      'Ind.T.Contr.Méd.H.Extras', 'Ind.T.Contr.Méd.V.Variav', 'Ind.Térm.Contr.Antec.Col.',
      'Ind.Térm.Contr.Antec.Emp', 'Insalubridade 13o Inden.', 'Insalubridade 13o Proporc',
      'Insalubridade Férias', 'Insalubridade Férias Resc', 'INSS Férias','Líquido Rescisão',
      'Média Adic.Noturno Féria', 'Média DSR 13o Prop.', 'Média DSR Férias', 'Média DSR Férias Abono Pe',
      'Média DSR. 13o Ind.', 'Média DSR. Férias Resc', 'Média Fer.Trab. 13o Prop.',
      'Média Fer.Trab.Férias Res', 'Média H.Extra Férias', 'Média H.Extras 13o Inden',
      'Média H.Extras 13o Prop.', 'Média HE. Férias Resc', 'Média Hs.Ref. 13o Ind.',
      'Média Hs.Ref. 13o Prop.', 'Média V.Variáv.Férias Res', 'Pensão Judicial 13o Sal.',
      'Pensão Judicial Férias', 'Peric.Férias Rescisão', 'Periculosidade 13o Inden.',
      'Periculosidade 13o Prop.', 'Periculosidade Férias', 'Recesso Férias Proporc.Re', 'Troco Férias' ]

  folha = folha[ (~folha['Empregador'].isin(condonal) ) | (~folha['Desc_Evento'].isin(remover) ) ]



  return folha

In [ ]:
# Limpa os arquivos no diretório
def clear(html=True, xlsx=True):
  files = os.listdir()

  if html:
    for file in files:
      if '.html' in file or '.HTML' in file:
        os.remove(file)

  if xlsx:
    for file in files:
      if '.xlsx' in file or '.xls' in file:
        os.remove(file)

## CERUS, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, SINGULAR, UNICA E VIGON

#### Processamento

In [ ]:
# Esté método funciona para folhas de pagamento das empresas dos grupos:
# CERUS, FIBRA, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, UNICA e VIGON
def get_registros_from_html(path:str, nome_arq='FolhaPagamento') -> pd.DataFrame:

  # Abre o arquivo e remove informações desnecessárias do arquivo
  arq_html = clean_html(path)

  # Declaração de variáveis
  columns = ['Competência','Empregador', 'CNPJ', 'Código_Empregado','Nome_Empregado','Cargo', 'Admissão','Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  # Encontra o CNPJ e Empregador que são únicos para o arquivo
  for i, row in enumerate(arq_html):
    if row ==  'Empregador':
      # O empregador está uma linha após a palavra
      empregador = arq_html[i+1]
      cnpj = str(arq_html[i-1])
      # Remove alguns itens do CNPJ para padronização
      remover = ['CNPJ: ', '.', '/', '-']
      for item in remover:
        cnpj = cnpj.replace(item, '')
      break

  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'Data e Assinatura':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in tqdm_notebook(range(total_func), leave=False):

    # A coleta destas informações não é feita por uso de um 'for'
    # pois a depender da empresa, alguma informação pode mudar
    # de posição

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Competência':
        # A competência está uma linha após esta palavra
        competencia = arq_html[i+1]
        break
    # Encontra a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Lotação':
        # A lotação está uma linha após a palavra
        lotacao = arq_html[i+1]
        break
    # Encontra o cargo
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Cargo':
        # O cargo está uma linha após a palavra
        cargo = arq_html[i+1]
        break
    # Encontra a admissão
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Admissão':
        # A admissão está uma linha após a palavra
        admissao = arq_html[i+1]
        break
    # Encontra o código e nome do empregado
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Empregado':
        # O código e nome estão uma linha após a palavra
        cod_e_nome = arq_html[i+1].split()
        codigo_emp = cod_e_nome[0]
        nome_emp = ' '.join(cod_e_nome[1:])
        break

    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'Cod.'
      if row == 'Cod.':
        index = i+1
        break


    while arq_html[index] != 'Total de Proventos':
      registro.append(arq_html[index])
      index += 1

      if len(registro) == 5:
        # Verifica se o código é não nulo
        if arq_html[index-1] or arq_html[index]:
          eventos.append(registro)
          registro = list()
        # Caso seja nulo, os eventos acabaram
        else:
          break

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Larga a coluna com as referências
      eventos = eventos.drop(2, axis='columns')
      # Inverte a ordem as colunas
      eventos = eventos[eventos.columns[::-1]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']
    else:
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([999, 'Afastado', 0, 0])

      index = len(registros)
      registros.loc[index] = registro


    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro


    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'Data e Assinatura':
        start = i
        if not nome_emp in arq_html[i+18]:
          break

  registros = padroniza(registros)

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

In [ ]:
# Atualiza alguns códigos para evitar conflitos
# Nem sempre é necessário
def update_cods(df:pd.DataFrame) -> pd.DataFrame:
  # Relação de códigos antigos e novos
  novo = {
      '350': '99A',
      '388': '99B'
  }

  for i in range(df.shape[0]):
    # É convertido para string pois alguns vem como int
    codigo = str(df['Código_Evento'].iloc[i])
    if codigo in novo:
      df.loc[i, 'Código_Evento'] = novo[codigo]

  return df

#### Leitura dos Arquivos

In [59]:
# Lista todos os arquivos html no diretório
files = [file for file in os.listdir() if '.html' in file]
folha_geral = pd.DataFrame()
# Lê cada arquivo no diretório e junta todos em um único DataFrame
for file in tqdm_notebook(files):
  folha_temp = get_registros_from_html(file)
  folha_geral = pd.concat([folha_geral, folha_temp])
# Converte o DataFrame para excel
folha_geral.to_excel('Folha_PRIMEE_PORTARIA.xlsx', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

## CONDONAL

#### Processamento

In [8]:
# Este método serve para as folhas do grupo CONDONAL
def get_registros_from_html_condonal(path:str, nome_arq='FolhaPagamento') -> pd.DataFrame:

  # Abre o arquivo e remove informações desnecessárias do arquivo
  arq_html = clean_html(path)

  # Declaração de variáveis
  columns = ['Competência','Empregador', 'CNPJ', 'Código_Empregado','Nome_Empregado','Cargo', 'Admissão', 'Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  # Função usada para ajustar o formato das competências
  def get_date(row):
    correspondencia = {
      '01/2023':   'Janeiro de 2023',
      '02/2023': 'Fevereiro de 2023',
      '03/2023':     'Março de 2023',
      '04/2023':     'Abril de 2023',
      '05/2023':      'Maio de 2023',
      '06/2023':     'Junho de 2023',
      '07/2023':     'Julho de 2023',
      '08/2023':    'Agosto de 2023',
      '09/2023':  'Setembro de 2023',
      '10/2023':   'Outubro de 2023',
      '11/2023':  'Novembro de 2023',
      '12/2023':  'Dezembro de 2023'
    }
    return correspondencia[row]



  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'EMPRESA':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in tqdm_notebook(range(total_func), leave=False):

    # Encontra o empregador e a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'EMPRESA':
        empregador = arq_html[i+6]
        cnpj = arq_html[i+7]
        lotacao = arq_html[i+8]
        break

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'SEQUÊNCIA':
        codigo_emp = arq_html[i+1]
        nome_emp = arq_html[i+2]
        admissao = arq_html[i+3]
        cargo = arq_html[i+4]
        competencia = arq_html[i+17]+arq_html[i+18]+arq_html[i+19]
        competencia = get_date(competencia)
        break


    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'BATIDAS'
      if row == 'BATIDAS':
        index = i+1
        break

    # Não há nenhuma palavra chave que indique o fim dos eventos
    # o fim dos eventos é seguido por uma série de linhas em branco
    # então é feita uma verificação das linhas em branco, se chegar
    # em 15, a coleta é encerrada
    count_nuls = 0
    # Pode acontecer de no meio dos eventos, ao fim de um evento a
    # página acabar, então a próxima linha será 'Continua...'
    # neste caso, os valores lidos devem ser ignorados até que se
    # encontre uma linha com o valor 'BATIDAS', que indica a
    # continuação dos eventos
    conteudo = True
    while count_nuls < 15:
      if arq_html[index] == 'Continua...':
        conteudo = False
        registro = list()

      if conteudo:
        if arq_html[index+1] == 'R E C I B O  D E  Q U I T A Ç Ã O':
          break
        registro.append(arq_html[index])
        if len(registro) == 14:
          eventos.append(registro)
          registro = list()
        # Contagem linhas em branco
        if arq_html[index]:
          count_nuls = 0
        else:
          count_nuls += 1
      else:
        if arq_html[index] == 'BATIDAS':
          conteudo = True
      index += 1

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Guarda apenas as colunas necessárias
      eventos = eventos[[3, 5, 9, 11]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']

    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro

    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'EMPRESA':
        start = i
        break

  registros = padroniza(registros)

  registros = registros.drop_duplicates()
  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

#### Leitura dos Arquivos

In [ ]:
files = [file for file in os.listdir() if '.HTML' in file]
folha_geral = pd.DataFrame()

for file in tqdm_notebook(files):
  folha_temp = get_registros_from_html_condonal(file)
  folha_geral = pd.concat([folha_geral, folha_temp])

folha_geral.to_excel('Folha_CONDONAL_COMERCIO_20.xlsx', index=False)

## FIBRA

### Antigo

#### Processamento

In [9]:
# Este método lê os registros de folhas de pagamento de arquivos excel
# As empresas do grupo FIBRA entregam neste formato
def get_registros_from_xlsx_fibra(path:str, competencia:str, empregador:str, cnpj:str, start:int, end:int, indice:int) -> pd.DataFrame:
  # A Admissão vem no formato AAAA-MM-DD 00:00:00, está funcão deixa na forma DD/MM/AAAA
  def ajusta_admissao(data):
    return f'{data[8:10]}/{data[5:7]}/{data[:4]}'

  columns = ['Competência',	'Empregador',	'CNPJ', 'Código_Empregado',	'Nome_Empregado',	'Cargo', 'Admissão',	'Lotação',	'Código_Evento',	'Desc_Evento',	'Provento',	'Desconto']
  registros = pd.DataFrame(columns=columns)

  columns = ['Numero', 'Nome', 'Jorn.', 'Cargo', 'Admissão', 'SF', 'IR']
  nrows = end - start + 2
  arq = pd.read_excel(path, skiprows=start, nrows=nrows).fillna('')
  arq.columns = columns

  admissao = str()
  nome = str()
  codigo = str()
  cargo = str()
  index = 1
  registro = list()

  lotacao = arq['Numero'].iloc[0]
  # Percorre todos os usuários, o fim dos dados de cada usuários
  # é seguido do código do próximo
  while arq['Numero'].iloc[index][1:].isnumeric():
    eventos = pd.DataFrame(columns=['Código', 'Descrição', 'Provento', 'Desconto'])

    admissao = str(arq['Admissão'].iloc[index])
    codigo = arq['Numero'].iloc[index]
    nome = arq['Nome'].iloc[index]
    cargo = arq['Cargo'].iloc[index]

    admissao = ajusta_admissao(admissao)

    # Encontra o começo dos eventos
    for i in range(index, nrows):
      if arq['Numero'].iloc[i] == 'Eventos':
        index = i+1
        break

    # Este while coleta todos os eventos
    # Os eventos são encerrados pela palavra 'Totais'
    while arq['Numero'].iloc[index] != 'Totais':
      registro = list()
      # Codigo evento
      registro.append(arq['Numero'].iloc[index])
      # Descrição
      registro.append(arq['Nome'].iloc[index])
      # Provento
      registro.append(arq['Admissão'].iloc[index])
      # Desconto
      registro.append(arq['SF'].iloc[index])

      eventos.loc[len(eventos)] = registro
      index += 1
    index += 1

    #Para cada linha de eventos, cria um registro completo
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, cnpj, codigo, nome, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])

      registros.loc[len(registros)] = registro


  registros = padroniza(registros)

  nome_arq = f'folha_parte_{indice:04}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

#### Leitura dos Arquivos

In [ ]:
empregador = 'FIBRA TECNOLOGIA'
cnpj = '43635413000155'
meses = {
    '012023.xls': 'Janeiro de 2023',
    '022023.xls': 'Fevereiro de 2023',
    '032023.xls': 'Março de 2023',
    '042023.xls': 'Abril de 2023',
    '052023.xls': 'Maio de 2023',
    '062023.xls': 'Junho de 2023',
    '072023.xls': 'Julho de 2023',
    '082023.xls': 'Agosto de 2023',
    '092023.xls': 'Setembro de 2023',
    '102023.xls': 'Outubro de 2023',
    '112023.xls': 'Novembro de 2023',
    '122023.xls': 'Dezembro de 2023'
}

In [ ]:
folha_completa = pd.DataFrame()
for mes in tqdm_notebook(meses):
  if mes in os.listdir():
    folha_fibra = pd.read_excel(mes).fillna('')
    competencia = meses[mes]
    start = int()
    indice = 0

    end = 0
    for i in tqdm_notebook(range(2, folha_fibra.shape[0]), leave=False):
      if folha_fibra['Numero'].iloc[i] == 'Totais':
        if not folha_fibra['Numero'].iloc[i+1][1:].isnumeric():
          start = end + 1
          end = i
          folha_parte = get_registros_from_xlsx_fibra(mes, competencia, empregador, cnpj, start, end, indice)
          indice += 1
          folha_completa = pd.concat([folha_completa, folha_parte])

folha_completa.to_excel(f'Folha_{empregador}.xlsx', index=False)

### Novo

#### Processamento

In [10]:
def get_registros_from_xlsx_fibra_novo(path:str) -> pd.DataFrame:

  folha = pd.read_excel(path).fillna('')

  if not 'Processo' in folha.columns:
    folha.columns = folha.iloc[0]
    if 'Lotação' in folha.columns:
      folha.rename(columns={'Lotação': 'Setor'}, inplace=True)

  folha = folha[(folha['Processo']=='Folha de Pagamento')&(folha['Natureza'] != 'Base')].reset_index(drop=True)

  folha = folha[['Mes', 'Empresa', 'Situação', 'Matricula', 'Nome',
                 'Cargo', 'Admissao', 'Setor', 'CODIGO', 'Evento',
                 'Natureza', 'Valor', 'Situação', 'Situação']]

  folha.columns = ['Competência', 'Empregador', 'CNPJ', 'Código_Empregado',
                   'Nome_Empregado', 'Cargo', 'Admissão', 'Lotação', 'Código_Evento',
                   'Desc_Evento', 'Tipo', 'Valor', 'Provento', 'Desconto']

  folha['CNPJ'] = [''] * folha.shape[0]
  folha['Provento'] = [''] * folha.shape[0]
  folha['Desconto'] = [''] * folha.shape[0]

  competencias = {
      '01/2023': 'Janeiro de 2023',
      '02/2023': 'Fevereiro de 2023',
      '03/2023': 'Março de 2023',
      '04/2023': 'Abril de 2023',
      '05/2023': 'Maio de 2023',
      '06/2023': 'Junho de 2023',
      '07/2023': 'Julho de 2023',
      '08/2023': 'Agosto de 2023',
      '09/2023': 'Setembro 2023',
      '10/2023': 'Outubro de 2023',
      '11/2023': 'Novembro de 2023',
      '12/2023': 'Dezembro de 2023',
      '01/2024': 'Janeiro de 2024',
  }

  cnpjs = {
      'Fibra Negocios e Servicos Ltda':            '02199192000132',
      'Fibra Tecnologia':                          '43635413000155',
      'Garra Escolta Vigilancia e Seguranca':      '04262215000131',
      'Tec Servicos e Seguranca Eletronica LTDA' : '39679787000103'
  }

  folha['Competência'] = folha['Competência'].apply(lambda x: competencias[x])

  folha['Empregador'] = folha['Empregador'].apply(lambda x: x[x.find(' - ')+3:].strip())

  folha['CNPJ'] = folha['Empregador'].apply(lambda x: cnpjs[x])

  folha['Lotação'] = folha['Lotação'].apply(lambda x: x[x.find(' - ')+3:])

  for i in range(folha.shape[0]):
    tipo = folha['Tipo'].iloc[i]
    folha.at[i, tipo] = folha['Valor'].iloc[i]


  folha = folha[['Competência', 'Empregador', 'CNPJ', 'Código_Empregado',
                 'Nome_Empregado', 'Cargo', 'Admissão', 'Lotação',
                 'Código_Evento', 'Desc_Evento', 'Provento', 'Desconto']]


  return folha

#### Leitura dos Arquivos

In [ ]:
folha = pd.DataFrame()
files = [file for file in os.listdir() if '.xlsx' in file]

for file in tqdm_notebook(files):
  folha_temp = get_registros_from_xlsx_fibra_novo(file)
  folha = pd.concat([folha, folha_temp])

folha.to_excel('Folha_FIBRA_TEC.xlsx', index=False)

## SOLUCAO

#### Processamento

In [11]:
# Resgata os registros de arquivos .xlsx de empresas do grupo SOLUCAO
# O formato como as planilhas são recebidos não é aberto com pd.read_excel()
# é necessário manualmente abrir a planilha e clicar em 'Formatar como tabela'
def get_registros_from_xlsx_solucao(path:str) -> pd.DataFrame:

  arq = pd.read_excel(path).fillna('')

  eventos = pd.DataFrame()
  columns = ['Competência',	'Empregador', 'CNPJ',	'Código_Empregado',	'Nome_Empregado',	'Cargo', 'Admissão', 'Lotação',	'Código_Evento',	'Desc_Evento',	'Provento',	'Desconto']
  registros = pd.DataFrame(columns=columns)

  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  index = 0
  start = 0
  end = int()
  proximo = bool()
  count_empregados = 0

  col_lotacao = str()
  col_descricoes = str()
  col_proventos = str()
  col_descontos = str()
  col_competencia = str()
  col_nome = str()

  # Definição das colunas que serão guardadas

  # Guarda o empregador que é um para o arquivo todo
  empregador = arq['Coluna3'].iloc[0]
  for coluna in arq.columns[4:]:
    if arq[coluna].iloc[1]:
      cnpj = arq[coluna].iloc[1]
      cnpj = f'{int(cnpj):014}'
      break

  # Encontra a coluna em que está a lotação
  for coluna in arq:
    if arq[coluna].iloc[4] == 'Departamento':
      col_lotacao = coluna
      break

  # Encontra a coluna em que estão as descricoes
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Descrição':
      col_descricoes = coluna
      break

  # Encontra a coluna em que estão os proventos
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Vencimentos':
      col_proventos = coluna
      break

  # Encontra a coluna em que estão os descontos
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Descontos':
      col_descontos = coluna
      break

  # Encontra a coluna em que estão as competências
  for i, coluna in enumerate(arq):
    if arq[coluna].iloc[1] == 'Folha Mensal':
      col_competencia = arq.columns[i+1]
      break

  # Encontra a coluna em que está o nome
  for coluna in arq:
    if arq[coluna].iloc[4] == 'Nome do Funcionário':
      col_nome = coluna
      break

  # Encontra a coluna em que está a admissão
  proximo = False
  for coluna in arq:
    if arq[coluna].iloc[6]:
      if proximo:
        col_admissao = coluna
        break
      elif arq[coluna].iloc[6] == 'Admissão:':
        proximo = True

  # Guarda apenas as colunas necessárias e as renomeia
  colunas = ['Coluna2', col_descricoes, col_proventos, col_descontos, col_nome, col_admissao, col_lotacao, col_competencia]
  arq = arq[colunas]
  arq.columns = ['Código', 'Descrição', 'Provento', 'Desconto', 'Nome_Empregado', 'Admissão', 'Lotação', 'Competência']

  # Conta o total de empregados na folha

  for i in arq['Código'].tolist():
    if i == 'Código':
      count_empregados += 1

  # Início da coleta de dados

  for _ in tqdm_notebook(range(count_empregados), leave=False):

    # Encontra a palavra chave 'Código' que será chave para achar os outros dados
    start = index
    for i, row in enumerate(arq['Código'].tolist()[start:], start=start):
      if row == 'Código':
        index = i+1
        break
    # Coleta os outros dados que estão 'ao redor' da palavra 'Código'
    codigo_emp = arq['Código'].iloc[index-4]
    nome_emp = arq['Nome_Empregado'].iloc[index-4]
    cargo = arq['Nome_Empregado'].iloc[index-3]
    admissao = str(arq['Admissão'].iloc[index-3])
    lotacao = arq['Lotação'].iloc[index-4]
    competencia = arq['Competência'].iloc[index-7]

    admissao = f'{admissao[8:10]}/{admissao[5:7]}/{admissao[:4]}'
    # Coleta os eventos

    end = index
    # Procura o fim dos eventos
    while arq['Código'].iloc[end]:
      end += 1
    else:
      eventos = arq[['Código', 'Descrição', 'Provento', 'Desconto']].iloc[index:end]


    #Para cada linha de eventos, cria um registro completo
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])

      registros.loc[len(registros)] = registro



  return registros

In [12]:
def ajusta_lotacaoes(folha:str) -> pd.DataFrame:

  # Essa função foi criada pois na tabela de relações, o nome da empresa
  # em questão está incompleto, então é criada uma coluna nova na folha
  # com o nome do emregador, mas no caso de ser a empresa com o nome cortado,
  # será adicionado o nome cortado
  def ajusta_empregador(emp):
    if emp == 'MAIS SOLUCAO EM LOCACAO DE MAO DE OBRA INTEGRADA S.A.':
      return ' MAIS SOLUCAO EM LOCACAO DE MAO DE OBRA I'
    else:
      return emp


  folha = folha.drop_duplicates().fillna('').reset_index()
  # Cria uma coluna auxiliar com o nome dos empregadores pois o nome
  # de alguns na tabela de relaççoes é diferente
  folha['Empregador_temp'] = folha['Empregador']
  folha['Empregador_temp'] = folha['Empregador_temp'].apply(ajusta_empregador)

  relacoes = pd.read_excel('Tabela de departamento.xlsx').fillna('')

  #folha = ajusta_codigos_eventos(folha, relacoes)

  for i in tqdm_notebook(range(folha.shape[0])):
    empresa = folha['Empregador_temp'].iloc[i]
    codigo_lot = folha['Lotação'].iloc[i]

    nome_codigo = relacoes['Nome'][(relacoes['Empresa']==empresa) & (relacoes['Código']==int(codigo_lot))].tolist()[0]

    folha.at[i, 'Lotação'] = nome_codigo

  # Larga a coluna Empregador_temp
  folha = folha.drop('Empregador_temp', axis='columns')

  return folha

#### Leitura dos Arquivos

In [ ]:
folha = pd.DataFrame()

files = [file for file in os.listdir() if 'Recibo' in file]

for file in tqdm_notebook(files):
  folha_temp = get_registros_from_xlsx_solucao(file)
  folha = pd.concat([folha, folha_temp])

folha = ajusta_lotacaoes(folha)
folha = padroniza(folha)
folha.to_excel('Folha_GRUPO_SINGULAR.xlsx', index=False)

## FOLK

In [45]:
def padroniza_folk(folha:pd.DataFrame) -> pd.DataFrame:

  descs = {
      'ADICIONAL 1/3 S/FERIAS':'ADICIONAL 1/3 S/FERIAS',
      'ADICIONAL ASSIDUIDADE':'ADICIONAL ASSIDUIDADE',
      'ADICIONAL DE PERICULOSIDADE 30,00%':'ADICIONAL DE PERICULOSIDADE',
      'ADICIONAL NOTURNO 20% 103:41':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 104:00':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 104:11':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 39:30':'ADICIONAL NOTURNO 20%',
      'AUXILIO COMBUSTIVEL':'AUXILIO COMBUSTIVEL',
      'Afastado':'Afastado',
      'BOLSA AUXILIO':'BOLSA AUXILIO',
      'BONIFICAÇÃO':'BONIFICAÇÃO',
      'CO PARTICIPAPAÇÃO A. ODONTO':'CO PARTICIPAPAÇÃO A. ODONTO',
      'CO PARTICIPAÇÃO A. MÉDICA':'CO PARTICIPAÇÃO A. MÉDICA',
      'COMPL. INT. H.E. 13 SAL. 1/12':'COMPL. INT. H.E. 13 SAL.',
      'COMPL. INT. VENC. VAR. 13 SAL. 1/12':'COMPL. INT. VENC. VAR. 13 SAL.',
      'COMISSOES':'COMISSOES',
      'DEDUCAO INSS EM IRRF FERIAS':'DEDUCAO INSS EM IRRF FERIAS',
      'DESC REFEICAO VINCULADA AO PAT':'DESC REFEICAO VINCULADA AO PAT',
      'DESC. ADIANTAMENTO DE SALARIO':'DESC. ADIANTAMENTO DE SALARIO',
      'DESC. INSS COMPLEMENTO 13º':'DESC. INSS COMPLEMENTO 13º',
      'DESCONTO DE ATRASOS 16:36':'DESCONTO DE ATRASOS',
      'DESCONTO DE VALE TRANSPORTE 6,00%':'DESCONTO DE VALE TRANSPORTE',
      'DESCONTO I.R.R.F. 15,00%':'DESCONTO I.R.R.F.',
      'DESCONTO I.R.R.F. 27,50%':'DESCONTO I.R.R.F.',
      'DESCONTO I.R.R.F. 7,50%':'DESCONTO I.R.R.F.',
      'DESCONTO INSS':'DESCONTO INSS',
      'DIAS AFASTAMENTO 01/30':'DIAS AFASTAMENTO',
      'FARMÁCIA':'FARMÁCIA',
      'FERIAS NORMAIS':'FERIAS NORMAIS',
      'GRATIFICAÇÃO':'GRATIFICAÇÃO',
      'HORA EXTRA 050% 00:15':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:30':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:13':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:46':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:21':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:30':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:37':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:07':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:52':'HORA EXTRA 050%',
      'HORA EXTRA 050% 04:58':'HORA EXTRA 050%',
      'HORA EXTRA 050% 82:25':'HORA EXTRA 050%',
      'HORA EXTRA 100% 00:36':'HORA EXTRA 100%',
      'HORA EXTRA 100% 01:31':'HORA EXTRA 100%',
      'HORA EXTRA 100% 04:06':'HORA EXTRA 100%',
      'HORA EXTRA 100% 04:19':'HORA EXTRA 100%',
      'HORA EXTRA 100% 08:42':'HORA EXTRA 100%',
      'HORA EXTRA 100% 09:39':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:25':'HORA EXTRA 100%',
      'ADICIONAL NOTURNO 20% 65:30':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 65:30':'ADICIONAL NOTURNO 20%',
      'HORA EXTRA 050% 01:04':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:04':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:42':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:42':'HORA EXTRA 050%',
      'HORA EXTRA 050% 04:59':'HORA EXTRA 050%',
      'HORA EXTRA 050% 04:59':'HORA EXTRA 050%',
      'HORA EXTRA 050% 08:56':'HORA EXTRA 050%',
      'HORA EXTRA 050% 08:56':'HORA EXTRA 050%',
      'HORA EXTRA 100% 00:11':'HORA EXTRA 100%',
      'HORA EXTRA 100% 00:11':'HORA EXTRA 100%',
      'HORA EXTRA 100% 04:12':'HORA EXTRA 100%',
      'HORA EXTRA 100% 04:12':'HORA EXTRA 100%',
      'HORA EXTRA 100% 08:23':'HORA EXTRA 100%',
      'HORA EXTRA 100% 08:23':'HORA EXTRA 100%',
      'INTEGRACAO HORA EXTRA NO DSR 05/21':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 05/21':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 05/21':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 05/21':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'SALARIO NORMAL 25/30':'SALARIO NORMAL',
      'SALARIO NORMAL 25/30:':'SALARIO NORMAL',
      'HORAS ATESTADO 00:40':'HORAS ATESTADO',
      'I.R.R.F. FERIAS':'I.R.R.F. FERIAS',
      'INT. ADIC. NOTURNO FERIAS':'INT. ADIC. NOTURNO FERIAS',
      'INT. H.E. FERIAS':'INT. H.E. FERIAS',
      'INT. VENC. VARIAVEIS FERIAS':'INT. VENC. VARIAVEIS FERIAS',
      'INTEGRACAO COMISSOES NO DSR 06/24':'INTEGRACAO COMISSOES NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 02/03':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 06/23':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 06/24':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 06/25':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 02/03':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 06/24':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 06/25':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'LIQUIDO FERIAS NORMAIS':'LIQUIDO FERIAS NORMAIS',
      'PENSAO ALIMENTICIA SALARIO':'PENSAO ALIMENTICIA SALARIO',
      'PRO-LABORE SEM FGTS':'PRO-LABORE SEM FGTS',
      'PRÊMIO':'PRÊMIO',
      'SALARIO NORMAL 05/30':'SALARIO NORMAL',
      'SALARIO NORMAL 29/30':'SALARIO NORMAL',
      'SALARIO NORMAL 30/30':'SALARIO NORMAL',
      'ADICIONAL NOTURNO 20% 00:09':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 00:22':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 00:37':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 00:45':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 06:29':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 103:42':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 103:44':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 104:14':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 105:06':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 105:07':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 107:06':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 13:00':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 19:55':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 59:00':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 97:56':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 98:54':'ADICIONAL NOTURNO 20%',
      'ARREDONDAMENTO MES':'ARREDONDAMENTO MES',
      'ARREDONDAMENTO MES ANTERIOR':'ARREDONDAMENTO MES ANTERIOR',
      'DESCONTO DE ATRASOS 00:11':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 05:36':'DESCONTO DE ATRASOS',
      'DESCONTO DE FALTAS 12x36 1':'DESCONTO DE FALTAS 12x36',
      'DESCONTO DE FALTAS 12x36 10':'DESCONTO DE FALTAS 12x36',
      'DESCONTO DE FALTAS 12x36 2':'DESCONTO DE FALTAS 12x36',
      'DESCONTO DE FALTAS 12x36 3':'DESCONTO DE FALTAS 12x36',
      'DESCONTO DE FALTAS 12x36 5':'DESCONTO DE FALTAS 12x36',
      'DESCONTO DE FALTAS INTEGRAIS 180:00':'DESCONTO DE FALTAS INTEGRAIS',
      'DESCONTO DESCANSO SEM. REMUNER 40:00':'DESCONTO DESCANSO SEM. REMUNER',
      'DESCONTO DSR 12X36 1':'DESCONTO DSR 12X36',
      'DESCONTO DSR 12X36 2':'DESCONTO DSR 12X36',
      'DESCONTO DSR 12X36 4':'DESCONTO DSR 12X36',
      'DESCONTO I.R.R.F. 22,50%':'DESCONTO I.R.R.F.',
      'DEVOLUÇÃO DESC. INDEVIDO':'DEVOLUÇÃO DESC. INDEVIDO',
      'DIAS AFASTAMENTO 13/30':'DIAS AFASTAMENTO',
      'DIAS AFASTAMENTO 15/30':'DIAS AFASTAMENTO',
      'HORA EXTRA 050% 00:10':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:11':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:12':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:13':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:14':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:25':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:27':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:29':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:33':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:47':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:50':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:56':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:58':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:05':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:06':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:14':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:32':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:34':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:48':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:28':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:36':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:40':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:41':'HORA EXTRA 050%',
      'HORA EXTRA 050% 11:08':'HORA EXTRA 050%',
      'HORA EXTRA 100% 00:09':'HORA EXTRA 100%',
      'HORA EXTRA 100% 00:13':'HORA EXTRA 100%',
      'HORA EXTRA 100% 00:31':'HORA EXTRA 100%',
      'HORA EXTRA 100% 01:17':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:16':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:21':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:23':'HORA EXTRA 100%',
      'HORA EXTRA 100% 10:32':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:28':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:29':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:30':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:31':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:32':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:33':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:34':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:36':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:38':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:40':'HORA EXTRA 100%',
      'HORA EXTRA 100% 12:33':'HORA EXTRA 100%',
      'HORA EXTRA 100% 12:34':'HORA EXTRA 100%',
      'HORA EXTRA 100% 12:46':'HORA EXTRA 100%',
      'HORA EXTRA 100% 13:15':'HORA EXTRA 100%',
      'I.R.R.F. 13 SALARIO COMPL. 15,00%':'I.R.R.F. 13 SALARIO COMPL.',
      'I.R.R.F. 13 SALARIO COMPL. 22,50%':'I.R.R.F. 13 SALARIO COMPL.',
      'I.R.R.F. 13 SALARIO COMPL. 7,50%':'I.R.R.F. 13 SALARIO COMPL.',
      'INTEGRACAO COMISSOES NO DSR 03/14':'INTEGRACAO COMISSOES NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 04/14':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 05/25':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 04/14':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 05/19':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'SALARIO FAMILIA 1':'SALARIO FAMILIA 1',
      'SALARIO FAMILIA 2':'SALARIO FAMILIA 2',
      'SALARIO MATERNIDADE 16/30':'SALARIO MATERNIDADE',
      'SALARIO NORMAL 01/30':'SALARIO NORMAL',
      'SALARIO NORMAL 03/30':'SALARIO NORMAL',
      'SALARIO NORMAL 04/30':'SALARIO NORMAL',
      'SALARIO NORMAL 10/30':'SALARIO NORMAL',
      'SALARIO NORMAL 11/30':'SALARIO NORMAL',
      'SALARIO NORMAL 14/30':'SALARIO NORMAL',
      'SALARIO NORMAL 17/30':'SALARIO NORMAL',
      'SALARIO NORMAL 19/30':'SALARIO NORMAL',
      'SALARIO NORMAL 23/30':'SALARIO NORMAL',
      'SALARIO NORMAL 24/30':'SALARIO NORMAL',
      'SALARIO NORMAL 26/30':'SALARIO NORMAL',
      'SALARIO NORMAL 27/30':'SALARIO NORMAL',
      'ADICIONAL NOTURNO 20% 00:07':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 00:10':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 106:12':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 110:43':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 111:22':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 85:40':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 89:38':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 96:49':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 97:15':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 97:43':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 97:45':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 98:00':'ADICIONAL NOTURNO 20%',
      'ADICIONAL NOTURNO 20% 99:36':'ADICIONAL NOTURNO 20%',
      'DESCONTO DE ATRASOS 00:13':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 00:14':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 00:22':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 00:32':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 01:06':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 01:13':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 01:25':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 02:07':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 05:28':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 05:52':'DESCONTO DE ATRASOS',
      'DESCONTO DE ATRASOS 30:31':'DESCONTO DE ATRASOS',
      'DIAS AFASTAMENTO 02/30':'DIAS AFASTAMENTO',
      'DIAS AFASTAMENTO 03/30':'DIAS AFASTAMENTO',
      'HORA EXTRA 050% 00:02':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:04':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:22':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:23':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:28':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:36':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:38':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:40':'HORA EXTRA 050%',
      'HORA EXTRA 050% 00:57':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:00':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:16':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:26':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:30':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:31':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:35':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:54':'HORA EXTRA 050%',
      'HORA EXTRA 050% 01:57':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:00':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:01':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:02':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:20':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:26':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:27':'HORA EXTRA 050%',
      'HORA EXTRA 050% 02:38':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:01':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:23':'HORA EXTRA 050%',
      'HORA EXTRA 050% 03:25':'HORA EXTRA 050%',
      'HORA EXTRA 050% 04:14':'HORA EXTRA 050%',
      'HORA EXTRA 050% 04:41':'HORA EXTRA 050%',
      'HORA EXTRA 050% 05:49':'HORA EXTRA 050%',
      'HORA EXTRA 050% 05:59':'HORA EXTRA 050%',
      'HORA EXTRA 050% 06:01':'HORA EXTRA 050%',
      'HORA EXTRA 050% 08:04':'HORA EXTRA 050%',
      'HORA EXTRA 050% 09:54':'HORA EXTRA 050%',
      'HORA EXTRA 050% 19:43':'HORA EXTRA 050%',
      'HORA EXTRA 050% 22:20':'HORA EXTRA 050%',
      'HORA EXTRA 050% 33:03':'HORA EXTRA 050%',
      'HORA EXTRA 050% 34:56':'HORA EXTRA 050%',
      'HORA EXTRA 050% 36:18':'HORA EXTRA 050%',
      'HORA EXTRA 100% 00:04':'HORA EXTRA 100%',
      'HORA EXTRA 100% 03:34':'HORA EXTRA 100%',
      'HORA EXTRA 100% 04:00':'HORA EXTRA 100%',
      'HORA EXTRA 100% 05:50':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:00':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:19':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:20':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:24':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:26':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:30':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:43':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:45':'HORA EXTRA 100%',
      'HORA EXTRA 100% 07:49':'HORA EXTRA 100%',
      'HORA EXTRA 100% 08:01':'HORA EXTRA 100%',
      'HORA EXTRA 100% 08:45':'HORA EXTRA 100%',
      'HORA EXTRA 100% 08:46':'HORA EXTRA 100%',
      'HORA EXTRA 100% 09:13':'HORA EXTRA 100%',
      'HORA EXTRA 100% 09:22':'HORA EXTRA 100%',
      'HORA EXTRA 100% 10:54':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:26':'HORA EXTRA 100%',
      'HORA EXTRA 100% 11:35':'HORA EXTRA 100%',
      'INTEGRACAO HORA EXTRA NO DSR 03/13':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 04/19':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 05/14':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 05/24':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRACAO HORA EXTRA NO DSR 06/22':'INTEGRACAO HORA EXTRA NO DSR',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR 06/23':'INTEGRAÇÃO ADIC.NOTURNO NO DSR',
      'SALARIO MATERNIDADE 30/30':'SALARIO MATERNIDADE',
      'SALARIO NORMAL 06/30':'SALARIO NORMAL',
      'SALARIO NORMAL 15/30':'SALARIO NORMAL',
      'SALARIO NORMAL 22/30':'SALARIO NORMAL',
      'SALARIO NORMAL 28/30':'SALARIO NORMAL'
  }

  tipos = {
      'ADICIONAL 1/3 S/FERIAS':'Provento',
      'ADICIONAL ASSIDUIDADE':'Provento',
      'ADICIONAL DE PERICULOSIDADE':'Provento',
      'ADICIONAL NOTURNO':'Provento',
      'AUXILIO COMBUSTIVEL':'Provento',
      'Afastado':'Provento',
      'BOLSA AUXILIO':'Provento',
      'BONIFICAÇÃO':'Provento',
      'CO PARTICIPAPAÇÃO A. ODONTO':'Desconto',
      'CO PARTICIPAÇÃO A. MÉDICA':'Desconto',
      'COMISSOES':'Provento',
      'DEDUCAO INSS EM IRRF FERIAS':'Provento',
      'DESC REFEICAO VINCULADA AO PAT':'Desconto',
      'DESC. ADIANTAMENTO DE SALARIO':'Desconto',
      'DESCONTO DE ATRASOS':'Desconto',
      'DESCONTO DE VALE TRANSPORTE':'Desconto',
      'DESCONTO I.R.R.F.':'Desconto',
      'DESCONTO INSS':'Desconto',
      'DIAS AFASTAMENTO':'Provento',
      'FARMÁCIA':'Desconto',
      'FERIAS NORMAIS':'Provento',
      'GRATIFICAÇÃO':'Provento',
      'HORA EXTRA 050%':'Provento',
      'HORA EXTRA 100%':'Provento',
      'HORAS ATESTADO':'Provento',
      'I.R.R.F. FERIAS':'Desconto',
      'INT. ADIC. NOTURNO FERIAS':'Provento',
      'INT. H.E. FERIAS':'Provento',
      'INT. VENC. VARIAVEIS FERIAS':'Provento',
      'INTEGRACAO COMISSOES NO DSR':'Provento',
      'INTEGRACAO HORA EXTRA NO DSR':'Provento',
      'INTEGRAÇÃO ADIC.NOTURNO NO DSR':'Provento',
      'LIQUIDO FERIAS NORMAIS':'Desconto',
      'PENSAO ALIMENTICIA SALARIO':'Desconto',
      'PRO-LABORE SEM FGTS':'Provento',
      'PRÊMIO':'Provento',
      'SALARIO NORMAL':'Provento',
      'ADICIONAL NOTURNO 20%':'Provento',
      'COMPL. INT. H.E. 13 SAL.':'Provento',
      'COMPL. INT. VENC. VAR. 13 SAL.':'Provento',
      'DESC. INSS COMPLEMENTO 13º':'Desconto',
      'ARREDONDAMENTO MES':'Provento',
      'ARREDONDAMENTO MES ANTERIOR':'Desconto',
      'DESCONTO DE FALTAS 12x36':'Desconto',
      'DESCONTO DE FALTAS INTEGRAIS':'Desconto',
      'DESCONTO DESCANSO SEM. REMUNER':'Desconto',
      'DESCONTO DSR 12X36':'Desconto',
      'DEVOLUÇÃO DESC. INDEVIDO':'Provento',
      'I.R.R.F. 13 SALARIO COMPL.':'Desconto',
      'SALARIO FAMILIA 1':'Provento',
      'SALARIO FAMILIA 2':'Provento',
      'SALARIO MATERNIDADE':'Provento'
  }


  erros = list()
  for i in folha['Desc_Evento'].unique():
    if i not in descs:
      erros.append(i)
  if erros:
    print('Descrições faltantes:')
    display(sorted(erros))

  folha['Desc_Evento'] = folha['Desc_Evento'].apply(lambda x: descs[x])


  erros = list()
  for i in folha['Desc_Evento'].unique():
    if i not in tipos:
      erros.append(i)
  if erros:
    print('Tipos faltantes:')
    display(sorted(erros))


  folha['Provento'] = '' * len(folha)
  folha['Desconto'] = '' * len(folha)

  for i in range(folha.shape[0]):
    tipo = tipos[folha['Desc_Evento'].iloc[i] ]
    folha[tipo].iloc[i] = folha['Valor'].iloc[i]

  folha = folha[ [
      'Competência', 'CNPJ', 'Empregador', 'Código_Empregado',
      'Nome_Empregado', 'Cargo', 'Admissão', 'Lotação',
      'Código_Evento', 'Desc_Evento', 'Provento', 'Desconto' ] ]

  return folha

### HTML

#### Processamento

In [51]:
def get_registros_from_html_folk(path:str) -> pd.DataFrame:

  df = pd.read_excel(path).fillna('')
  arq = df['Unnamed: 0'].to_list()

  registros = list()

  empregador = arq[0]

  temp = arq[2].split()

  cnpj = temp[0]
  cnpj = cnpj.replace('.','').replace('/','').replace('-','')

  competencia = temp[-1]
  competencia = competencia.replace('/', ' de ')

  i = 0

  while i < len(arq)-1:

    temp = arq[i+3].split()

    cod_emp = temp[0]

    nome_emp = ' '.join(temp[1:-1])


    temp = arq[i+4].replace('Admissão: ', '').split()

    cargo = ' '.join(temp[:-1])

    admissao = temp[-1]

    lotacao = arq[i+6]

    i += 7

    if str(arq[i]).replace(',', '').replace('.', '').replace(' ', '').isnumeric():

      registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([999, 'Afastado', 0])
      registros.append(registro)

    while not str(arq[i]).replace(',','').replace('.','').replace(' ','').isnumeric():

      if arq[i] == 'CONTINUA':
        i += 9

      else:

        temp = arq[i].split()
        cod_evento = temp[0]
        desc_evento = ' '.join(temp[1:-1])
        valor = temp[-1]

        registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao, lotacao]
        registro.extend([cod_evento, desc_evento, valor])
        registros.append(registro)

        i += 1


    while i < len(arq) and (arq[i] != empregador or nome_emp in arq[i+3]):
      i += 1


  columns = ['Competência', 'Empregador', 'CNPJ', 'Código_Empregado',
             'Nome_Empregado', 'Cargo', 'Admissão', 'Lotação',
             'Código_Evento','Desc_Evento', 'Valor']
  registros = pd.DataFrame(registros, columns=columns)


  registros = padroniza_folk(registros)

  return registros

#### Leitura dos Arquivos

In [52]:
folha = get_registros_from_html_folk('folha_FOLK_112023.xlsx')

In [53]:
folha.to_excel('folha_FOLK_112023.xlsx', index=False)

### PDF

#### Processamento

In [25]:
def get_registros_from_pdf_folk(path:str) -> pd.DataFrame:

  df = pd.read_excel(path).fillna('')
  arq = df['Unnamed: 0'].to_list()

  registros = list()

  i = 12

  empregador = arq[i]

  temp = arq[i+2].split()

  cnpj = temp[0]
  cnpj = cnpj.replace('.','').replace('/','').replace('-','')

  competencia = arq[i+3].split()[1]
  competencia = competencia.replace('/', ' de ')

  while i < len(arq)-1:

    temp = arq[i+4].split()

    cod_emp = temp[0]

    nome_emp = ' '.join(temp[1:-1])


    temp = arq[i+5].replace('Admissão: ', '').split()

    cargo = ' '.join(temp[:-1])

    admissao = temp[-1]

    lotacao = arq[i+7]

    i += 8

    if str(arq[i]).replace(',', '').replace('.', '').replace(' ', '').isnumeric():

      registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([999, 'Afastado', 0])
      registros.append(registro)

    while not str(arq[i]).replace(',','').replace('.','').replace(' ','').isnumeric():

      if arq[i] == 'CONTINUA':
        i += 10

      else:

        temp = arq[i].split()
        cod_evento = temp[0]
        desc_evento = ' '.join(temp[1:-1])
        valor = temp[-1]

        registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao, lotacao]
        registro.extend([cod_evento, desc_evento, valor])
        registros.append(registro)

        i += 1


    while i < len(arq) and (arq[i] != empregador or nome_emp in arq[i+4]):
      i += 1


  columns = ['Competência', 'Empregador', 'CNPJ', 'Código_Empregado',
             'Nome_Empregado', 'Cargo', 'Admissão', 'Lotação',
             'Código_Evento','Desc_Evento', 'Valor']
  registros = pd.DataFrame(registros, columns=columns)


  registros = padroniza_folk(registros)

  return registros

#### Leitura dos Arquivos

In [46]:
folha = get_registros_from_pdf_folk('folha_KIENEN_112023.xlsx')

In [47]:
folha.to_excel('folha_KIENEN_112023.xlsx', index=False)

# Junção de Arquivos

In [63]:
folha = pd.DataFrame()

files = [file for file in os.listdir() if '.xlsx' in file]

for file in tqdm_notebook(files):
  folha_temp = pd.read_excel(file)

  folha = pd.concat([folha, folha_temp])

folha = padroniza(folha)
folha.to_excel('Folha_GERAL.xlsx', index=False)

  0%|          | 0/13 [00:00<?, ?it/s]

# -

In [62]:
clear()